# Chiral solutions for Type-II Dirac seesaw with a Dark Symmetry

In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)

## Methodology
1. __Apply__ the conditions and store the list of dictionaries with the solutions in column 'tree_level'. For each dictionary add:
   * List of massless quiral fermions
   * Best extra escalar, `sp` and list of massless fermions after use `sp` → `sp_massless`
2. __Filter__ the solutions with empty `sp_massless` list
3. __Prepre__ equivalent solutions. Creates new columns with the counting of each used particle
5. __Prepare__ output

General functions

In [3]:
from filter_functions import *

## Load full solutions

In [4]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json')
ds.shape

(148097, 5)

Check that solutions are unique

In [5]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(148097, 6)

In [6]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [7]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

95358

## 1. Check phenomenological conditions
$$ \nu+m+s=0$$

In [8]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [9]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    triplet=False
    for x in ll:
        if ll.count(x)>1:
            if ll.count(x)>2:
                triplet=True
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2 and triplet:
        MR=True
    return MR



#assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
#assert best_higgs_singlet([1,-11,9])==(2, [])
#assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]
#assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None

In [10]:
def remove_majorana(l,s):
    #Remove Majorana
    xx=(np.unique(l)*2-s)
    xx=(xx[xx!=0]+s)/2
    xx=(np.unique(xx)*2+s)
    xx=(xx[xx!=0]-s)/2
    return xx

def check_rank(l,ν):
    massless=list(set([x for x in  l if x!=ν ]))
    s=-ν
    return rank(l,massless,s)

def extract_massless(l,ν):
    '''
    Return 0 if the integers in massless appears as s-paired same-dimension multiplets in l, 1 otherwise.
    s-paired means that exist two intengers in massless such that n1+n2=±s. (n1 can be equal to n2)
    
    For example, for l containing only the massless=[1,2] integers:
        l=[1,2]; s=3 → 0: 3-paired multiplet: ([1],[2]) of 1-dimension multiplets (A.K.A singlet)
        l=[1,1,2]; s=3 → 1: unpaired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2]; s=3 → 0: 3-paired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2,2]; s=3 → 1: unpaired multiplets: ([1,1,1],[2,2]) doblet and singlet
    massless=[1,2,3]
    l=[1,2,3]; s=4 → 0: 4-paired multiplet: ([1],[3]) of singlets
                            4-paired multiplet: ([1],[3]) of singlet
    '''
    s=-ν
    #Keep massless
    m=[x for x in  l if x!=ν ]
    #m=[x for x in  m if m.count(x)!=1 ]
    mltps=list( set( [m.count(x) for x in m ]  ))
    #print(mltps)
    Maj=[]
    if len(mltps)>1:
        for x in set(m):
            for z in [y for y in set(m) if y!=x and abs(y+x)==abs(s) and abs(x)<abs(y) ]:
                if l.count(x)>l.count(z):
                    Maj.append(x)
                elif l.count(x)<l.count(z):
                    Maj.append(z)
    if len(Maj)==1:
        return Maj[0]*2,None
    elif len(Maj)>1:
        return Maj[0]*2,Maj[1:] 
    else:
        return None,None

    
def extract_sp_massless(l,massless,sp):
    #Restore full massless list
    m=[x for x in  l if x in massless ]
    #Obtain the multiplets
    mltps=list( set( [m.count(x) for x in m]  ))
    Maj=[]
    if len(mltps)>1:
        for x in set(m):
            for z in [y for y in set(m) if y!=x and abs(y+x)==sp and abs(x)<abs(y) ]:
                if l.count(x)>l.count(z):
                    Maj.append(x)
                elif l.count(x)<l.count(z):
                    Maj.append(z)
    return Maj
    


#******* rank        ****************
assert rank([1,2],[1,2],3)==0
assert rank([1,1,2,2,2],[1,2],3)==1
assert rank([1,2,3],[1,2,3],4)==0
assert rank([1, 1, -4, 9, -10, -10, 16, 16, -19],[9, -19, 1],18)==1
assert rank( [1, -4, -4, 9, 9, -11], [-11],22)==0
assert rank( [-16,-16,-16, 17, 17],[-16, 17],1)==1
assert rank( [-16,-16,-16, 17, 17,-18,-18,19,19,19],[-16, 17,-18,19],1)==0
#************************************
#********* extract_massless ************
assert extract_massless([1,-5, -6, 10, 10,10, 9,9,2,2,-11, -11],-11)==(20, None)
assert extract_massless([1,1,-5, -6, 10, 10,10, 9,9,2,2,-11, -11],-11)==(20, None)
assert extract_massless([1,1,-5, -6, 10, 10,10, 9,9,2,2,2,-11, -11],-11)==(20, [2])
assert extract_massless([1, 1, 1, -4, -4, 5],1)==(-8, None)
#******************************************
#********* extract_sp_massless ************
assert extract_sp_massless([1, 1, 1, 4, 4, 5, -10, -10, -10, 14],[1,5],6)==[1]
#******************************************

In [11]:
def tree_level_ii(ll):
    '''
    At least a triplet and a doublet is guaranteed
    (ν,ν,...)
    '''
    fs=[]
    νrp=[]
    for x in np.unique(ll):
        if ll.count(x)==2 or ll.count(x)==3:
            νrp.append(x)
    νrp=sorted(np.unique(νrp))
    #print(len(mrp),len(νrp),ll)
    sltn=[]
    for ν in νrp:
        s=-ν
        slt={'[ν,s]':[ν,s]}
        fs=list(np.unique([x for x in ll if x!=ν]))
        massless=extract_Dirac_and_Majorana(fs,s)
        if len(massless)>0:
            slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            if rank(ll,massless,slt['sp'])==1:
                slt['sp_massless']=slt['sp_massless']+extract_sp_massless(  ll,massless,slt['sp']  )
            if check_rank(ll,ν)==1:
                spextra,tmp=extract_massless(ll,ν)
                if spextra:
                    slt['sp_massless']=slt['sp_massless']+[spextra//2]
                

        else: # Not extra sp required
            if check_rank(ll,ν)==1:
                slt['sp'],slt['sp_massless']=extract_massless(ll,ν)
            else:
                slt['sp'],slt['sp_massless']=(None,None)            
        sltn.append(slt)
    #Extract best solution
    sltn0=[d for d in sltn if d.get('sp') is None]
    if len(sltn0)>0: #Solution without extra `sp`
        return sltn0[0]
    sltn0=[d for d in sltn if d.get('sp') and isinstance(d.get('sp_massless'),list) and len(d.get('sp_massless'))==0]
    if len(sltn0)>0: #Solution with extra `sp` empty sp_massless
        return sltn0[0]
    ##TODO: Add sp  but with None sp_massless
    sltn0=[d for d in sltn if d.get('sp') and d.get('sp_massless') is None]
    if len(sltn0)>0: #Solution with extra `sp` empty sp_massless
        return sltn0[0]
    return np.nan

assert tree_level_ii([1,1,-4,-5,9,9,9,-10,-10]).get('[ν,s]')==[9, -9]
assert tree_level_ii([1, 2, 2, -3, -3, -3, 4]) is np.nan #ν→ -3 [1,2,2]→ massless 2

### Initialize filtered solutions

In [12]:
cl=ds.copy()

Find solutions that satisfy the condition

In [13]:
cl['tree_level']=cl['solution'].apply(tree_level_ii)

## 2. Filter `sp_massless`

Remove solutions with `'sp_massless'` length > 0, i.e, with `'tree_level'=NaN`

In [14]:
cl=cl.dropna(subset=['tree_level']).reset_index(drop=True)
cl.shape

(8523, 6)

In [15]:
cl=cl[cl['n']<10].reset_index(drop=True)

In [16]:
cl.shape

(1038, 6)

In [17]:
cl[cl['tree_level'].apply(lambda d: d.get('sp') is None)].shape

(28, 6)

In [18]:
cl['sp']=cl['tree_level'].apply(lambda d: d.get('sp') )

In [19]:
cl['ν']=cl['tree_level'].apply(lambda d: d.get('[ν,s]')[0]  )
cl['sp']=cl['tree_level'].apply(lambda d: d.get('sp') )
cl['sp']=cl['sp'].fillna('None').apply(lambda x: int(x) if x!='None' else x)
cl['massless']=cl.apply(lambda row: extract_Dirac_and_Majorana
                       (
                        list(np.unique([x for x in row['solution'] if x!=row['ν'] ])),-row.get('ν')
                                         ),axis='columns')
#Fix massless for rank zero

Reorder the filtered solutions

In [20]:
cl[cl['solution'].astype(str)=='[1, 1, 2, 2, 3, -5, -6, -6, 8]']

,l,k,solution,gcd,n,tree_level,sp,ν,massless
944,"[-6, -3, -6, -2]","[-6, -4, -1, -5]","[1, 1, 2, 2, 3, -5, -6, -6, 8]",1,9,"{'[ν,s]': [2, -2], 'sp': -12, 'sp_massless': None}",-12,2,[]


## 3. Check the rank of the chiral fermion matrix 

Fix massless for rank o quiral fermion mass matrices

In [21]:
cl['rk']      =cl.apply(lambda row: 1              if row['sp']!='None' and row['massless']==[] else 0              ,axis='columns')

cl['massless']=cl.apply(lambda row: [row['sp']//2] if row['sp']!='None' and row['massless']==[] else row['massless'],axis='columns')

In [22]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl.sort_values(['n','smax']).reset_index(drop=True)
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level,sp,ν,massless,rk,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"{'[ν,s]': [1, -1], 'sp': -8, 'sp_massless': None}",-8,1,[-4],1,5


## 4. Flag equivalent solutions

In [23]:
def characterize_solution(cl,i,j=0):
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    cl.loc[i,'trptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>2] )
    cl.loc[i,'frptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>3] )
    return cl

Example: massless solutions

In [24]:
for i in cl.index:
    cl=characterize_solution(cl,i,0)

In [25]:
cl['nsp']=cl['tree_level'].apply(lambda d: 0 if d.get('sp') is None else 1)

### Add solutions with massless fermions that get masses with some $S'$

In [26]:
for k  in ['nsp','rptd','trptd','frptd']:
    cl[k]=cl[k].astype(int)

In [27]:
cl=cl.sort_values(['n','smax','nsp','rptd','trptd','frptd']).reset_index(drop=True)

## Build the final table dropping out equivalent solutions 

In [31]:
fl=pd.DataFrame()
for n in range(6,10):
    print(n)
    fl=fl.append( cl[ cl['n']==n ].drop_duplicates(subset=['nsp','rptd','trptd','frptd','rk']).reset_index(drop=True) )

6
7
8
9


## 5. Prepare output

In [33]:
def getit(it,dk):
    it['ν']=dk.get('[ν,s]')[0]
    it['s']=dk.get('[ν,s]')[1]
    return it
    
tm=pd.DataFrame()
fl=fl.sort_values(['n','smax']).reset_index(drop=True)
for i in fl.index:
    it=fl.loc[i].to_dict()
    k=fl.loc[i,'tree_level']
    dk=k
    it=getit(it,dk)
    tm=tm.append(it,ignore_index=True)

In [34]:
for k in ['n','gcd','ν','s','rptd','trptd']:
    tm[k]=tm[k].astype(int)
tm['nsmax']=tm.apply(lambda row: np.abs( row['solution'] ).max(),axis='columns' )
tm=tm.sort_values(['n','nsmax']).reset_index(drop=True)

#Delete equivalent solutions
#tm['Q']=True
#sltns={'[3, -4, 8, -9]': [3, -4, -6, -6, 7, 7, 8, -9],}
#for k in sltns.keys():
#    iq=tm[((tm['solution'].astype(str)==str(sltns[k])) &  (tm['massless'].astype(str)==k)
#          )].index[0] #unstable dark matter
#    tm.loc[iq,'Q']=False

#tm=tm[tm['Q']].reset_index(drop=True).drop('Q',axis='columns')
tm[['n','l','k','solution','gcd','ν','s','massless','rptd','trptd']
  ].sort_values(['n','rptd','trptd'])#.loc[[0,4]]#.loc[[0,4]]

,n,l,k,solution,gcd,ν,s,massless,rptd,trptd
1,6,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,[],1,0
2,6,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,1,-1,"[-11, 8]",1,0
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-1,[-4],2,1
3,7,"[-1, 1]","[-1, -2, -1]","[1, 3, -4, 5, -6, -6, 7]",1,-6,6,"[-4, 7]",1,0
4,7,"[-1, 0, -2]","[-1, 0, -1]","[1, 7, 8, -9, -9, -9, 11]",1,-9,9,"[7, 11]",1,1
6,8,"[3, 2, -2]","[-7, -10, -4]","[2, -3, -4, 5, -6, 7, 7, -8]",1,7,-7,"[-8, -6]",1,0
5,8,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,2,-2,[-6],1,1
7,8,"[1, -2, 2]","[-1, 0, -3, 5]","[4, -5, -5, 7, 8, -10, -10, 11]",1,-10,10,"[4, 7, 8, 11]",2,0
8,8,"[-2, -1, 2]","[-2, -5, 1]","[1, 2, 2, -7, -7, 10, 10, -11]",2,10,-10,"[-7, 2]",3,0
11,9,"[-3, -1, 5]","[-9, 3, -4, -1]","[1, 2, -3, 4, -5, -6, 8, 8, -9]",1,8,-8,"[-6, 2]",1,0


In [35]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless','sp','rptd','trptd']].reset_index(drop=True)
kk['sltn']=kk['solution'].astype(str)
kk.sort_values('n').drop('sltn',axis='columns').reset_index(drop=True)

,n,l,k,solution,gcd,ν,s,massless,sp,rptd,trptd
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-1,[-4],-8,2,1
1,6,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,[],None,1,0
2,6,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,1,-1,"[-11, 8]",3,1,0
3,7,"[-1, 1]","[-1, -2, -1]","[1, 3, -4, 5, -6, -6, 7]",1,-6,6,"[-4, 7]",3,1,0
4,7,"[-1, 0, -2]","[-1, 0, -1]","[1, 7, 8, -9, -9, -9, 11]",1,-9,9,"[7, 11]",18,1,1
5,8,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,2,-2,[-6],12,1,1
6,8,"[3, 2, -2]","[-7, -10, -4]","[2, -3, -4, 5, -6, 7, 7, -8]",1,7,-7,"[-8, -6]",14,1,0
7,8,"[1, -2, 2]","[-1, 0, -3, 5]","[4, -5, -5, 7, 8, -10, -10, 11]",1,-10,10,"[4, 7, 8, 11]",15,2,0
8,8,"[-2, -1, 2]","[-2, -5, 1]","[1, 2, 2, -7, -7, 10, 10, -11]",2,10,-10,"[-7, 2]",5,3,0
9,9,"[-4, -5, 3]","[-2, 0, -1, -2]","[3, 3, -4, 5, 5, -6, -8, -8, 10]",1,-8,8,"[-6, 10]",4,3,0


In [36]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless','sp']].copy()

In [37]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldskymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [38]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless','sp']].copy()

for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if not kk.loc[i,'massless']:
        print(i)
        kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
        #kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])

kkk=kk#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

1
10
13


In [39]:
cat solutions.tex

\begin{tabular}{llllrrrll}
\toprule
              n &                 l &                 k &                           solution & gcd &   ν &   s &       massless &    sp \\
\midrule
              6 &          (-1, -2) &           (-1, 2) &               (1, 1, 1, -4, -4, 5) &   1 &   1 &  -1 &           (-4) &    -8 \\
 \boldsymbol{6} &           (-1, 1) &           (-2, 0) &              (1, -2, -3, 5, 5, -6) &   1 &   5 &  -5 &             () &  None \\
              6 &            (0, 2) &       (-1, -2, 1) &            (1, 1, 8, -11, -16, 17) &   1 &   1 &  -1 &       (-11, 8) &     3 \\
              7 &           (-1, 1) &      (-1, -2, -1) &           (1, 3, -4, 5, -6, -6, 7) &   1 &  -6 &   6 &        (-4, 7) &     3 \\
              7 &       (-1, 0, -2) &       (-1, 0, -1) &          (1, 7, 8, -9, -9, -9, 11) &   1 &  -9 &   9 &        (7, 11) &    18 \\
              8 &       (-1, 2, -2) &        (-7, 4, 0) &        (1, 2, 2, 2, -3, -5, -6, 7) &   1 &   2 &  -2 &         

In [40]:
kkk

,n,l,k,solution,gcd,ν,s,massless,sp
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-1,[-4],-8
1,\boldsymbol{6},"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,[],None
2,6,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,1,-1,"[-11, 8]",3
3,7,"[-1, 1]","[-1, -2, -1]","[1, 3, -4, 5, -6, -6, 7]",1,-6,6,"[-4, 7]",3
4,7,"[-1, 0, -2]","[-1, 0, -1]","[1, 7, 8, -9, -9, -9, 11]",1,-9,9,"[7, 11]",18
5,8,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,2,-2,[-6],12
6,8,"[3, 2, -2]","[-7, -10, -4]","[2, -3, -4, 5, -6, 7, 7, -8]",1,7,-7,"[-8, -6]",14
7,8,"[1, -2, 2]","[-1, 0, -3, 5]","[4, -5, -5, 7, 8, -10, -10, 11]",1,-10,10,"[4, 7, 8, 11]",15
8,8,"[-2, -1, 2]","[-2, -5, 1]","[1, 2, 2, -7, -7, 10, 10, -11]",2,10,-10,"[-7, 2]",5
9,9,"[-6, -3, -6, -2]","[-6, -4, -1, -5]","[1, 1, 2, 2, 3, -5, -6, -6, 8]",1,2,-2,[-6],-12


In [41]:
assert kkk.shape==(19, 9)